In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 13.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 18.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.6/197.6 kB 9.3 MB/s eta 0:00:00


In [2]:
import transformers

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
# DistilBERT:

## задаем саму модель
model_class = transformers.DistilBertModel

## токенайзер к ней (для некоторых моделей токенайзер будет отличаться, см.
## в документации к каждой модели конкретно)
tokenizer_class = transformers.DistilBertTokenizer

## загружаем веса для моделей
pretrained_weights = 'distilbert-base-uncased'



# AlBERT:
# model_class, tokenizer_class, pretrained_weights = (transformers.AlbertModel, 
#                                                     transformers.AlbertTokenizerFast,
#                                                     'albert-base-v1')

# BERT
# model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, 
#                                                     transformers.BertTokenizer, 
#                                                     'bert-base-uncased')




###########################################
# создаем объекты токенизатора для и модели
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
max_len = 100

In [4]:
import numpy as np
import pandas as pd

In [7]:
df = pd.DataFrame(pd.Series("Last week, I watched seasons 1-7 of GoT so those episodes are still very fresh in my mind and I am unaffected by rose tinted nostalgia. I have just watched episode 1 of House of the Dragon. How can I sum it up? So far, so good. The music is sufficiently different to be fresh while remaining recognizable. The production quality and cinematography are as expected - fantastic, epic and awe inspiring, aside from a couple of excessively dark scenes that were over and done with very quickly. There is gore aplenty. Nudity, vomit, sex and depravity. Incidental background humor. A promise of things to come. The casting seems to be on point - I saw no issue with any of the characters, and the leads were well chosen for their roles. The writing seems to be up to par. As an introduction, this episode was written and directed well. I want to see and know more - a good sign. So far I am pleased with this return to the land of Westeros. Valar morghulis!", name = 'review'))

In [8]:
df

,review
0,"Last week, I watched seasons 1-7 of GoT so tho..."


In [11]:
tokenized = df['review'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, 
                                                                      truncation=True, 
                                                                      max_length=max_len)))

In [12]:
tokenized

0    [101, 2197, 2733, 1010, 1045, 3427, 3692, 1015...
Name: review, dtype: object

In [13]:
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [14]:
attention_mask = np.where(padded != 0, 1, 0)

In [15]:
import torch

In [16]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

In [17]:
input_ids

tensor([[  101,  2197,  2733,  1010,  1045,  3427,  3692,  1015,  1011,  1021,
          1997,  2288,  2061,  2216,  4178,  2024,  2145,  2200,  4840,  1999,
          2026,  2568,  1998,  1045,  2572, 24720,  2011,  3123, 25577, 26968,
          1012,  1045,  2031,  2074,  3427,  2792,  1015,  1997,  2160,  1997,
          1996,  5202,  1012,  2129,  2064,  1045,  7680,  2009,  2039,  1029,
          2061,  2521,  1010,  2061,  2204,  1012,  1996,  2189,  2003, 12949,
          2367,  2000,  2022,  4840,  2096,  3588, 20123,  1012,  1996,  2537,
          3737,  1998, 16434,  2024,  2004,  3517,  1011, 10392,  1010,  8680,
          1998, 15180, 18988,  1010,  4998,  2013,  1037,  3232,  1997, 11664,
          2135,  2601,  5019,  2008,  2020,  2058,  1998,  2589,  2007,   102]])

In [18]:
with torch.inference_mode():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,0,:].numpy()

In [20]:
features.shape

(1, 768)

In [21]:
import joblib
from sklearn.linear_model import LogisticRegression

In [22]:
from sklearn.linear_model import LinearRegression


lr = joblib.load('LReg.pkl')


/Users/dbushuev/miniforge3/envs/neiro/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [24]:
lr.predict_proba(features)

array([[0.02422911, 0.97577089]])

In [26]:
!pip install catboost

  Using cached catboost-1.0.6-cp310-none-macosx_10_6_universal2.whl (22.0 MB)
  Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)
  Using cached plotly-5.10.0-py2.py3-none-any.whl (15.2 MB)
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)


In [27]:
from catboost import CatBoostClassifier 

In [28]:
cat = joblib.load('CatBC.pkl')
cat.predict_proba(features)

array([[0.0195976, 0.9804024]])